## 简答题
1. 如果已经在完全相同的训练集上训练了5个不同的模型，并且它们都达到了95%的准确率，是否还有机会获得更好的效果？如果可以，该怎么做？ 如果不行，为什么？
仍有机会：把 5 个 95 % 的模型再用投票/平均等策略集成，误差就会相互抵消，整体准确率通常高于 95 %。

2. 硬投票分类器和软投票分类器有什么区别？
硬投票看“谁票多”，只数类别标签；软投票看“谁信心高”，把各模型的预测概率加权平均后再决定类别。

3. 是否可以通过在多个服务器上并行来加速bagging集成的训练？pasting集成呢？提升集成呢？随机森林或堆叠集成呢？
可以并行。pasting集成，提升集成等大部分集成都不影响，boosting 必须串行迭代，无法直接并行。

4. 包外评估的好处是什么？
包外评估用未参与每棵树训练的样本做验证，相当于“自带验证集”，省掉额外交叉验证，且不会浪费数据。

5. 是什么让极端随机树比一般随机森林更加随机？这部分增加的随机性有什么用？极端随机树比一般随机森林快还是慢？
极端随机树不仅随机选取特征，还会随机取阈值。更多随机性进一步降低方差、加速训练，因此比常规随机森林更快。

6. 如果AdaBoost集成对训练数据欠拟合，应该调整哪些超参数？怎么调整？
增大基模型复杂度、增加n_estimators，减小learning_rate。

7. 如果梯度提升集成对训练集过拟合，应该提升还是降低学习率？
降低学习率。

## 编程题
1. 加载MNIST数据集（**8_sklearn做分类** 里有介绍），将其分为一个训练集、一个验证集和一个测试集（例如使用40000个实例训练，10000个实例验证，最后20000个实例测试）。然后训练多个分类器，比如一个随机森林分类器、一个极端随机树分类器和一个SVM。接下来，尝试使用软投票法或者硬投票法将它们组合成一个集成，这个集成在验证集上的表现要胜过它们各自单独的表现。成功找到集成后，在测试集上测试。与单个的分类器相比，它的性能要好多少？


2. 运行上一题中的单个分类器，用验证集进行预测，然后用预测结果创建一个新的训练集：新训练集中的每个实例都是一个向量，这个向量包含所有分类器对于一个图像的一组预测，目标值是图像的类。恭喜，你成功训练了一个混合器，结合第一层的分类器，它们一起构成了一个堆叠集成。现在在测试集上评估这个集成。对于测试集中的每个图像，使用所有的分类器进行预测，然后将预测结果提供给混合器，得到集成的预测。与前面训练的投票分类器相比，这个集成的结果如何？现在再次尝试使用StackingClassifier。你得到了更好的性能吗？如果是这样，为什么？

In [27]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [5]:
print("Loading MNIST ...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)


Loading MNIST ...


In [32]:
X = X.astype(np.float32) / 255.0
y = y.astype(np.int64)

In [33]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

In [34]:
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

et_clf = ExtraTreesClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

svm_clf  = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', C=10, gamma=0.01,random_state=42))
])



In [ ]:
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf), ('svm', svm_clf)],
    voting='hard',
    n_jobs=-1
)

models = {
    '随机森林' : rf_clf,
    '极端树'   : et_clf,
    '向量机'   : svm_clf,
    '软投票'   : voting_clf
}

for name,model in models.items():
    model.fit(X_train, y_train)
    y_pred =model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    print(f"{name} 验证集准确率: {score:.4f}")


In [ ]:
estimators=[rf_clf, et_clf, svm_clf]

X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [ ]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

In [ ]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [ ]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [ ]:
accuracy_score(y_test, y_pred)